# **Text Preprocessing**


In [10]:
import re
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from tokenizers import Tokenizer

In [11]:
def clean_text(text):
    char_map = {
        'ñ': 'n', 'ā': 'a', 'ī': 'i', 'ū': 'u', 'ḳ': 'k', 'ġ': 'g', 'Ḍ': 'D', 'D': 'D', 'ṣ': 's', 'ż': 'z', 'ḷ': 'l',
        'ḥ': 'h', 'ṭ': 't', 'ḍ': 'd', 'ś': 's', 'ṁ': 'm', 'ṣ': 's', 'é': 'e', 'ó': 'o'
    }

    # Replace special characters
    for special_char, normal_char in char_map.items():
        text = text.replace(special_char, normal_char)

    # Remove dots, commas, quotes, and question marks
    text = re.sub(r'[.,"\'?]', '', text)

    # Remove extra spaces but preserve new lines
    text = re.sub(r'[ \t]+', ' ', text).strip()

    return text

# Read CSV file
df = pd.read_csv('Roman-Urdu-Poetry.csv')

# Apply cleaning function to the 'Poetry' column
df['Poetry'] = df['Poetry'].astype(str).apply(clean_text)

# Print cleaned dataframe
print(df.loc[0, 'Poetry'])
# Save the cleaned DataFrame to a new CSV file
df.to_csv('Cleaned_Roman_Urdu_Poetry.csv', index=False)

# Download the file to your PC
# from google.colab import files
# files.download('Cleaned_Roman_Urdu_Poetry.csv')



aankh se duur na ho dil se utar jaega 
vaqt ka kya hai guzarta hai guzar jaega 
itna manus na ho khalvat-e-gham se apni 
tu kabhi khud ko bhi dekhega to Dar jaega 
Dubte Dubte kashti ko uchhala de duun 
main nahin koi to sahil pe utar jaega 
zindagi teri ata hai to ye jaane vaala 
teri bakhshish tiri dahliz pe dhar jaega 
zabt lazim hai magar dukh hai qayamat ka faraz 
zalim ab ke bhi na roega to mar jaega


# **Tokenization**

In [ ]:
from collections import Counter

# Character Pair Tokenizer
def char_pair_tokenizer(text, vocab_size=5000):
    # Initialize vocabulary with unique characters
    vocab = Counter(text)

    # Merge character pairs based on frequency until reaching vocab size
    while len(vocab) < vocab_size:
        pairs = Counter([text[i:i+2] for i in range(len(text)-1)])
        if not pairs:
            break
        most_common_pair = pairs.most_common(1)[0][0]
        text = text.replace(most_common_pair, f'{most_common_pair}')  # Merge the pair
        vocab.update([most_common_pair])

    # Create token-to-ID mapping
    token_to_id = {token: idx for idx, token in enumerate(vocab.keys())}
    return token_to_id, text

# Tokenize the entire dataset
all_text = ' '.join(df['Poetry'].tolist())
token_to_id, tokenized_text = char_pair_tokenizer(all_text)
# Character Tokenizer
def char_tokenizer(text):
    vocab = sorted(set(text))
    token_to_id = {char: idx + 1 for idx, char in enumerate(vocab)}  # Reserve 0 for padding
    token_to_id['<PAD>'] = 0
    return token_to_id

# Tokenize the entire dataset
all_text = ' '.join(df['Poetry'].dropna().tolist())
token_to_id = char_tokenizer(all_text)
vocab_size = len(token_to_id)

# Encode Poetry
def encode_poetry(poem):
    return [token_to_id.get(char, 0) for char in poem]  # 0 for unknown characters

poetry_data = df['Poetry'].dropna().tolist()
encoded_poetry = [encode_poetry(poem) for poem in poetry_data]


Original: dil se mohabbat
Encoded IDs: [2, 89, 54, 324, 3]
Tokens: ['<BOS>', 'dil', 'se', 'mohabbat', '<EOS>']


# Dataset Preparation for LSTM

In [ ]:


# Use the already cleaned DataFrame 'df'
poetry_data = df['Poetry'].dropna().tolist()

# Load the trained BPE tokenizer
tokenizer = Tokenizer.from_file("bpe_tokenizer.json")

# Encode Poetry
def encode_poetry(poem):
    tokens = tokenizer.encode("<BOS> " + poem + " <EOS>")
    return tokens.ids

encoded_poetry = [encode_poetry(poem) for poem in poetry_data]

# Dataset Class
class ShayariDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        seq = self.data[idx]
        return torch.tensor(seq[:-1]), torch.tensor(seq[1:])  # Input-Target pair

# Custom Collate Function for Padding
def collate_fn(batch):
    inputs, targets = zip(*batch)
    inputs_padded = pad_sequence(inputs, batch_first=True, padding_value=0)  # 0 as PAD token
    targets_padded = pad_sequence(targets, batch_first=True, padding_value=0)
    return inputs_padded, targets_padded

# DataLoader with Custom Collate Function
dataset = ShayariDataset(encoded_poetry)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, drop_last=True, collate_fn=collate_fn)

# Test a Batch
for inputs, targets in dataloader:
    print("Input Batch Shape:", inputs.shape)
    print("Target Batch Shape:", targets.shape)
    print("Sample Input IDs:", inputs[0])
    print("Sample Target IDs:", targets[0])
    break


Input Batch Shape: torch.Size([32, 394])
Target Batch Shape: torch.Size([32, 394])
Sample Input IDs: tensor([   2,  482, 1183, 2545,   54,  222,  357,   68,   54, 1405, 1405,   54,
         222,   64, 2429,   57,   72,   20,   39,  291,  119,  210,  999,   68,
          59,  155,  273,   54,  222, 3469,  515,  745, 1018,  270,   83,  193,
         149,   86,  209,   77,   78, 1736,   54,  222,  290,   44,  723, 2197,
          72,   55,   68,   59,  830,   44,  348,   63, 3234,  222,  107,   63,
        3455,   59,  152,   79,  743,   68,   59, 1038,   94,  269,   54,  222,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,

# LSTM Model Implementation

In [ ]:
import torch
import torch.nn as nn

# LSTM Model for Roman Urdu Shayari Generation
class ShayariLSTM(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_layers, dropout=0.2):
        super(ShayariLSTM, self).__init__()

        # Embedding Layer
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)

        # LSTM Layer
        self.lstm = nn.LSTM(embed_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout)

        # Fully Connected Output Layer
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, hidden):
        x = self.embedding(x)               # Embedding
        output, hidden = self.lstm(x, hidden)  # LSTM
        output = self.fc(output)               # Output Layer
        return output, hidden

    def init_hidden(self, batch_size, hidden_dim, num_layers, device):
        # Initialize hidden and cell states with zeros
        hidden = (torch.zeros(num_layers, batch_size, hidden_dim).to(device),
                  torch.zeros(num_layers, batch_size, hidden_dim).to(device))
        return hidden

# Model Hyperparameters
vocab_size = tokenizer.get_vocab_size()
embed_dim = 256
hidden_dim = 512
num_layers = 2
dropout = 0.3
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Model Initialization
model = ShayariLSTM(vocab_size, embed_dim, hidden_dim, num_layers, dropout).to(device)

# Display Model Architecture
print(model)


ShayariLSTM(
  (embedding): Embedding(5000, 256, padding_idx=0)
  (lstm): LSTM(256, 512, num_layers=2, batch_first=True, dropout=0.3)
  (fc): Linear(in_features=512, out_features=5000, bias=True)
)


In [ ]:
#Actual Volabualry size using BPE
vocab_size = tokenizer.get_vocab_size()
print(vocab_size)


5000


# Training the Model

In [ ]:
# Hyperparameters
num_epochs = 100
learning_rate = 0.001

# Loss Function and Optimizer
criterion = nn.CrossEntropyLoss(ignore_index=0)  # Ignore padding token (ID=0)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training Loop
for epoch in range(1, num_epochs + 1):
    model.train()
    hidden = model.init_hidden(batch_size=32, hidden_dim=512, num_layers=2, device=device)

    epoch_loss = 0
    epoch_correct = 0
    epoch_total = 0

    progress_bar = tqdm(dataloader, desc=f"Epoch {epoch}/{num_epochs}", leave=False)

    for inputs, targets in progress_bar:
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()

        # Detach hidden states to prevent backpropagating through the entire training history
        hidden = tuple(h.detach() for h in hidden)

        # Forward pass
        output, hidden = model(inputs, hidden)

        # Reshape output and target for loss calculation
        output = output.view(-1, output.size(2))  # [batch_size * sequence_length, vocab_size]
        targets = targets.view(-1)               # [batch_size * sequence_length]

        # Compute loss
        loss = criterion(output, targets)
        loss.backward()
        optimizer.step()

        # Calculate accuracy
        _, predicted = torch.max(output, dim=1)  # Get the index of the max log-probability

        # Ignore padding tokens when calculating accuracy
        mask = targets != 0                     # Mask to ignore padding tokens (ID=0)
        correct_predictions = (predicted[mask] == targets[mask]).sum().item()
        total_valid_tokens = mask.sum().item()

        # Update epoch statistics
        epoch_loss += loss.item()
        epoch_correct += correct_predictions
        epoch_total += total_valid_tokens

        # Display current loss and accuracy on progress bar
        progress_bar.set_postfix({
            'Loss': loss.item(),
            'Accuracy': f"{correct_predictions / total_valid_tokens:.4f}" if total_valid_tokens > 0 else "N/A"
        })

    # Calculate average loss and accuracy for the epoch
    avg_loss = epoch_loss / len(dataloader)
    epoch_accuracy = epoch_correct / epoch_total if epoch_total > 0 else 0

    print(f"Epoch {epoch}, Average Loss: {avg_loss:.4f}, Accuracy: {epoch_accuracy:.4f}")

Epoch 1, Average Loss: 6.4800, Accuracy: 0.1213


Epoch 2, Average Loss: 5.8989, Accuracy: 0.1578


Epoch 3, Average Loss: 5.7455, Accuracy: 0.1636


Epoch 4, Average Loss: 5.6359, Accuracy: 0.1720


Epoch 5, Average Loss: 5.5446, Accuracy: 0.1825


...

Epoch 96, Average Loss: 0.6970, Accuracy: 0.8487


Epoch 97, Average Loss: 0.6856, Accuracy: 0.8513


Epoch 98, Average Loss: 0.6698, Accuracy: 0.8543


Epoch 99, Average Loss: 0.6534, Accuracy: 0.8584


Epoch 100, Average Loss: 0.6398, Accuracy: 0.8612


In [5]:
# Text Generation Function
def generate_text(start_seq, gen_length=50):
    model.eval()
    tokens = tokenizer.encode(start_seq).ids
    input_seq = torch.tensor(tokens).unsqueeze(0).to(device)
    hidden = model.init_hidden(1, hidden_dim=512, num_layers=2, device=device)

    generated = tokens

    with torch.no_grad():
        for _ in range(gen_length):
            output, hidden = model(input_seq, hidden)
            prob = torch.softmax(output[:, -1, :], dim=-1)
            next_token = torch.argmax(prob, dim=-1).item()

            generated.append(next_token)
            input_seq = torch.tensor([[next_token]]).to(device)

    return tokenizer.decode(generated)

# Generate Sample Shayari
sample_shayari = generate_text("tujh pe uthi hain", gen_length=100)
print("\nGenerated Shayari:\n", sample_shayari)


NameError: name 'model' is not defined

# With Charater Pair Encoding

In [12]:
from collections import Counter
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

In [13]:
# Character Tokenizer
def char_tokenizer(text):
    vocab = sorted(set(text))
    token_to_id = {char: idx + 1 for idx, char in enumerate(vocab)}  # Reserve 0 for padding
    token_to_id['<PAD>'] = 0
    return token_to_id

# Tokenize the entire dataset
all_text = ' '.join(df['Poetry'].dropna().tolist())
token_to_id = char_tokenizer(all_text)
vocab_size = len(token_to_id)

# Encode Poetry
def encode_poetry(poem):
    return [token_to_id.get(char, 0) for char in poem]  # 0 for unknown characters

poetry_data = df['Poetry'].dropna().tolist()
encoded_poetry = [encode_poetry(poem) for poem in poetry_data]

# Dataset Class
class ShayariDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        seq = self.data[idx]
        return torch.tensor(seq[:-1]), torch.tensor(seq[1:])  # Input-Target pair

# Custom Collate Function for Padding
def collate_fn(batch):
    inputs, targets = zip(*batch)
    inputs_padded = pad_sequence(inputs, batch_first=True, padding_value=0)
    targets_padded = pad_sequence(targets, batch_first=True, padding_value=0)
    return inputs_padded, targets_padded

# DataLoader
dataset = ShayariDataset(encoded_poetry)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, drop_last=True, collate_fn=collate_fn)

# LSTM Model
class ShayariLSTM(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_layers, dropout=0.2):
        super(ShayariLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, hidden):
        x = self.embedding(x)
        output, hidden = self.lstm(x, hidden)
        output = self.fc(output)
        return output, hidden

    def init_hidden(self, batch_size, hidden_dim, num_layers, device):
        return (torch.zeros(num_layers, batch_size, hidden_dim).to(device),
                torch.zeros(num_layers, batch_size, hidden_dim).to(device))

# Model Initialization
embed_dim = 256
hidden_dim = 512
num_layers = 2
dropout = 0.3
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = ShayariLSTM(vocab_size, embed_dim, hidden_dim, num_layers, dropout).to(device)

# Training Loop
num_epochs = 100
learning_rate = 0.001
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(1, num_epochs + 1):
    model.train()
    hidden = model.init_hidden(batch_size=32, hidden_dim=hidden_dim, num_layers=num_layers, device=device)

    epoch_loss = 0
    epoch_correct = 0
    epoch_total = 0

    progress_bar = tqdm(dataloader, desc=f"Epoch {epoch}/{num_epochs}", leave=False)

    for inputs, targets in progress_bar:
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        hidden = tuple(h.detach() for h in hidden)

        output, hidden = model(inputs, hidden)
        output = output.view(-1, output.size(2))
        targets = targets.view(-1)

        loss = criterion(output, targets)
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(output, dim=1)
        mask = targets != 0
        correct_predictions = (predicted[mask] == targets[mask]).sum().item()
        total_valid_tokens = mask.sum().item()

        epoch_loss += loss.item()
        epoch_correct += correct_predictions
        epoch_total += total_valid_tokens

        progress_bar.set_postfix({
            'Loss': loss.item(),
            'Accuracy': f"{correct_predictions / total_valid_tokens:.4f}" if total_valid_tokens > 0 else "N/A"
        })

    avg_loss = epoch_loss / len(dataloader)
    epoch_accuracy = epoch_correct / epoch_total if epoch_total > 0 else 0
    print(f"Epoch {epoch}, Average Loss: {avg_loss:.4f}, Accuracy: {epoch_accuracy:.4f}")
    if epoch % 10 == 0:
        torch.save(model.state_dict(), f"shayari_lstm_epoch{epoch}.pth")
        torch.save(optimizer.state_dict(), f"optimizer_epoch{epoch}.pth")
        print(f"✅ Model saved at epoch {epoch}")

torch.save(model.state_dict(), "shayari_lstm_final.pth")
torch.save(optimizer.state_dict(), "optimizer_final.pth")
print("✅ Final model saved successfully!")

Epoch 1, Average Loss: 2.5046, Accuracy: 0.3100


Epoch 2, Average Loss: 1.9382, Accuracy: 0.4259


Epoch 3, Average Loss: 1.7655, Accuracy: 0.4681


Epoch 4, Average Loss: 1.6620, Accuracy: 0.4948


Epoch 5, Average Loss: 1.5920, Accuracy: 0.5132


...

Epoch 96, Average Loss: 0.6030, Accuracy: 0.8017


Epoch 97, Average Loss: 0.5976, Accuracy: 0.8034


Epoch 98, Average Loss: 0.5940, Accuracy: 0.8043


Epoch 99, Average Loss: 0.5893, Accuracy: 0.8055


Epoch 100, Average Loss: 0.5860, Accuracy: 0.8069
✅ Model saved at epoch 100
✅ Final model saved successfully!


In [9]:
# Load Model
model = ShayariLSTM(vocab_size, embed_dim, hidden_dim, num_layers, dropout).to(device)
model.load_state_dict(torch.load("shayari_lstm_final.pth"))  # Load saved weights
model.eval()  # Set model to evaluation mode

# Text Generation Function
def generate_text(start_seq, gen_length=50):
    model.eval()  # Ensure model is in evaluation mode
    tokens = [token_to_id.get(char, 0) for char in start_seq]
    input_seq = torch.tensor(tokens).unsqueeze(0).to(device)
    hidden = model.init_hidden(1, hidden_dim, num_layers, device)

    generated = tokens

    with torch.no_grad():
        for _ in range(gen_length):
            output, hidden = model(input_seq, hidden)
            prob = torch.softmax(output[:, -1, :], dim=-1)
            next_token = torch.argmax(prob, dim=-1).item()

            generated.append(next_token)
            input_seq = torch.tensor([[next_token]]).to(device)

    # Convert IDs back to characters
    id_to_token = {v: k for k, v in token_to_id.items()}
    return ''.join([id_to_token.get(token, '') for token in generated])

# Generate Sample Shayari
sample_shayari = generate_text("tujh pe uthi hain", gen_length=300)
print("\nGenerated Shayari:\n", sample_shayari)



Generated Shayari:
 tujh pe uthi hain sab ke sab khvab bhi ho gaya 
us ki tegh-e-nigah ko abhi tabassum uzr 
yuun to sab kuchh andaz-e-gul-e-tar ka taara 
ki jaise abru-e-asar-e-bahar se vo sun 
main apne aap men sar tha aur bulandi tu 
vasl bhi saaya surashon ka arman ho gaya hasil 
abhi faqirana na main nashsha-e-mai bahut thi 
main 
